In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wd/'

Mounted at /content/drive


In [3]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [4]:
filename = 'GKV_data.csv'
HV_data = pd.read_csv(path + filename)

In [5]:
set_index(HV_data)

In [6]:
tm_data = HV_data

In [7]:
HV_data = HV_data.drop(columns=['GKV'])

In [8]:
HV_data['GKV'] = tm_data['GKV']

In [9]:
HV_data

,"('GARCH', 'normal', 0)","('GARCH', 'gaussian', 0)","('GARCH', 'ged', 0)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)","('FIGARCH', 'gaussian', 0)","('FIGARCH', 'gaussian', 1)","('FIGARCH', 'ged', 0)","('FIGARCH', 'ged', 1)",gdp_growth,...,usd_eur,usd_jpy,usd_gbp,usd_cny,usd_cad,usd_mxn,gt_data,log_returns,"('APARCH', 'studentst', 1)",GKV
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-02,0.039601,0.039601,0.040044,0.038388,0.038388,0.038388,0.038388,0.038865,0.038865,28984.929215,...,2.781970,307.252039,4.529620,23.415205,4.090860,26.750733,209.303607,-0.005604,0.037128,0.038001
2000-02-03,0.037979,0.037979,0.041000,0.038633,0.038633,0.038633,0.038633,0.040342,0.040342,28984.929215,...,2.799436,311.491144,4.529266,23.412659,4.119569,26.561052,209.303607,-0.008693,0.038336,0.038737
2000-02-04,0.036791,0.036791,0.038214,0.036990,0.036990,0.036990,0.036990,0.039265,0.039265,28984.929215,...,2.728548,306.262089,4.474254,23.415452,4.106629,26.433950,209.303607,0.000572,0.049778,0.036803
2000-02-07,0.046593,0.046593,0.047968,0.046474,0.046474,0.046474,0.046474,0.048168,0.048168,28984.929215,...,2.733357,299.491542,4.460147,23.414391,4.140641,26.339020,209.303607,0.027307,0.065732,0.041112
2000-02-08,0.049131,0.049131,0.049560,0.046711,0.046711,0.046711,0.046711,0.045562,0.045562,28984.929215,...,2.730104,301.761354,4.470789,23.415982,4.142338,26.135551,209.303607,0.010478,0.043373,0.043562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,0.000120,0.000120,0.000456,0.000121,0.000121,0.000121,0.000121,0.000423,0.000423,0.000000,...,-0.002546,0.509117,-0.003041,0.005162,0.001980,-0.105925,0.404061,0.008246,-0.004076,0.000010
2024-08-27,-0.003889,-0.003889,-0.003539,-0.004368,-0.004368,-0.004368,-0.004368,-0.001972,-0.001972,0.000000,...,0.002051,-0.551543,-0.001202,-0.006788,-0.001556,-0.173948,0.404061,0.009336,-0.005604,0.000016
2024-08-28,0.000280,0.000280,0.000449,0.000483,0.000483,0.000483,0.000483,0.000381,0.000381,0.000000,...,0.000849,0.247487,0.000849,0.002263,0.002051,-0.219910,0.707107,0.010307,-0.001642,0.000028


# Split train, validation and test data

In [10]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [11]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [12]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [13]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## GKV

In [14]:
daily_train, daily_val, daily_test = split_data(HV_data, 'GKV')

In [15]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [16]:
HV_X_train, HV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
HV_X_val, HV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
HV_X_test, HV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [17]:
# HV_X_train = HV_X_train.reshape(HV_X_train.shape[0], win_sz, -1)
# HV_X_val = HV_X_val.reshape(HV_X_val.shape[0], win_sz, -1)
# HV_X_test = HV_X_test.reshape(HV_X_test.shape[0], win_sz, -1)

### LSTM

In [18]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │          15,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,605 (76.58 KB)

 Trainable params: 19,605 (76.58 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
lstm_history = lstm_garch_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - loss: 0.0052 - val_loss: 3.5263e-06
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 9.1371e-05 - val_loss: 2.1156e-06
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 5.6235e-05 - val_loss: 2.0297e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 5.1343e-05 - val_loss: 1.9127e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 6.5572e-05 - val_loss: 9.5050e-07
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 4.2637e-05 - val_loss: 7.6866e-07
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 4.2112e-05 - val_loss: 5.4383e-07
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 3.4438e-05 - val_loss: 3.3653e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 4.8883e-05 - val_loss: 2.1290e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 4.0479e-05 - val_loss: 1.2679e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step

In [21]:
# Predictions
HV_lstm_predictions = lstm_garch_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step


In [22]:
mse = mean_squared_error(HV_y_test, HV_lstm_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_lstm_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_lstm_predictions)
print(f'MAPE: {mape}%')

MSE: 1.310503481792864e-06
RMSE: 0.001144772240139
MAE: 0.0009346461270967199
MAPE: 4816.242057145283%


### GRU

In [23]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [24]:
gru_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 25, 48)              │          11,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,917 (58.27 KB)

 Trainable params: 14,917 (58.27 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
gru_history = gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - loss: 0.0581 - val_loss: 5.3096e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 2.2022e-04 - val_loss: 2.5474e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - loss: 1.1215e-04 - val_loss: 1.2448e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 9.9121e-05 - val_loss: 1.1844e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 7.8494e-05 - val_loss: 5.2123e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 7.4812e-05 - val_loss: 7.0223e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - loss: 6.5898e-05 - val_loss: 6.8335e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - loss: 4.9665e-05 - val_loss: 4.3744e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - loss: 5.4553e-05 - val_loss: 1.7385e-05
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 5.8043e-05 - val_loss: 9.1701e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/ste

In [26]:
# Predictions
HV_gru_predictions = gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step


In [27]:
mse = mean_squared_error(HV_y_test, HV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.0001104761461300087
RMSE: 0.010510763346684613
MAE: 0.010264768216601642
MAPE: 55299.48398514326%


### LSTM_GRU

In [28]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
lstm_gru_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 25, 48)              │          15,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,613 (72.71 KB)

 Trainable params: 18,613 (72.71 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
lstm_gru_history = lstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 0.0307 - val_loss: 8.1479e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 5.1222e-04 - val_loss: 3.7453e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 2.2493e-04 - val_loss: 2.6457e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 1.4574e-04 - val_loss: 9.6360e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 9.8874e-05 - val_loss: 6.6108e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 8.0458e-05 - val_loss: 3.2313e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 6.7462e-05 - val_loss: 6.5421e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 5.4460e-05 - val_loss: 6.5041e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - loss: 5.9201e-05 - val_loss: 1.8722e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - loss: 5.5766e-05 - val_loss: 3.3951e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step

In [31]:
# Predictions
HV_lstm_gru_predictions = lstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


In [32]:
mse = mean_squared_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 1.6381513483511432e-05
RMSE: 0.004047408242753804
MAE: 0.003500125115568589
MAPE: 18013.229927403612%


In [ ]:
# path_1 = '/content/drive/MyDrive/Volatility/Predictions/GKV/'

# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)

### BiLSTM

In [33]:
bilstm_model = Sequential()
bilstm_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Bidirectional(LSTM(16)))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Dense(5))

bilstm_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [34]:
bilstm_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 25, 96)              │          30,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 32)                  │          14,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 45,349 (177.14 KB)

 Trainable params: 45,349 (177.14 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
bilstm_history = bilstm_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 15s 59ms/step - loss: 0.0204 - val_loss: 2.4246e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 2.3264e-04 - val_loss: 1.1585e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step - loss: 1.2120e-04 - val_loss: 1.3884e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - loss: 8.5057e-05 - val_loss: 7.9195e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - loss: 6.7930e-05 - val_loss: 4.7005e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - loss: 6.8794e-05 - val_loss: 3.9556e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - loss: 5.8056e-05 - val_loss: 3.7649e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - loss: 6.0226e-05 - val_loss: 3.0227e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - loss: 4.8827e-05 - val_loss: 1.8730e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - loss: 4.4436e-05 - val_loss: 3.6342e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/st

In [36]:
# Predictions
HV_bilstm_predictions = bilstm_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step


In [37]:
mse = mean_squared_error(HV_y_test, HV_bilstm_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_predictions)
print(f'MAPE: {mape}%')

MSE: 2.920206282825307e-06
RMSE: 0.0017088611069438343
MAE: 0.0015443707659039748
MAPE: 8114.266401350993%


### BiLSTM_GRU

In [38]:
bilstm_gru_model = Sequential()
bilstm_gru_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(GRU(16))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(Dense(5))

bilstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [39]:
bilstm_gru_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional)      │ (None, 25, 96)              │          30,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 16)                  │           5,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 36,277 (141.71 KB)

 Trainable params: 36,277 (141.71 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
bilstm_gru_history = bilstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 51ms/step - loss: 0.0377 - val_loss: 9.8413e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 6.0158e-04 - val_loss: 3.1106e-04
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - loss: 2.4453e-04 - val_loss: 6.5286e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - loss: 1.1900e-04 - val_loss: 5.4335e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - loss: 8.6325e-05 - val_loss: 6.8798e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - loss: 6.8343e-05 - val_loss: 4.3394e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - loss: 5.9605e-05 - val_loss: 5.1837e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 5.2267e-05 - val_loss: 2.0835e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - loss: 4.3697e-05 - val_loss: 1.0731e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - loss: 4.4080e-05 - val_loss: 2.1906e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/s

In [41]:
# Predictions
HV_bilstm_gru_predictions = bilstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step


In [42]:
mse = mean_squared_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 1.9067278396861663e-06
RMSE: 0.0013808431625952915
MAE: 0.001042754585018026
MAPE: 5223.201067396922%


In [43]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/GKV/'

# df_pred = pd.DataFrame(HV_lstm_predictions[:,0], columns=['lstm_pred'])
# filename = 'LSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_gru_predictions[:,0], columns=['gru_pred'])
# filename = 'GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_predictions[:,0], columns=['bilstm_pred'])
# filename = 'BiLSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_gru_predictions[:,0], columns=['bilstm_gru_pred'])
# filename = 'BiLSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)